# Vid2Blog

This notebook was instigated by [a xeet posted by Andrej Karpathy](https://twitter.com/karpathy/status/1760740503614836917). Throughout this notebook, we will attempt to take a longform video and automatedly transform it into a blog post.

## Strategy

The strategy for enabling this will evolve over time, but here is what I am thinking as of right now:

- Use the OpenAI Whisper API to transcribe the audio from the video.
- Use LLM to break the transcription into logical segments.
- Determine the timestamp chunks from the previous step's segmentated script.
- Use something (there's gotta be something!) to automatedly break the video into chunks.
- Extract frames at regular intervals and save them to disk for later use (This is tough to get right...)
- Use LLM to summarize the segmented into an outline **in JSON form**. (Important to get it in a structured form for the next steps.)
- In parallel, produce each segment of the blog post by passing in the following info in each parallel call:
    - The full outline (Maybe this needs converted back from JSON into plaintext?)
    - The transcribed text from that particular section
    - GPT-V uses the images to give a better description of what's going on (Again, hard problem)
- Run the full blog post back through the LLM for "final clean up", ensuring cohesion and proper attributions are moved to the end of the post

## Notebook Setup
Let's do our imports and such!

In [1]:
import os
import json
import yaml
from langchain_openai import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate

In [3]:
# Loading my personal OpenAI API key
with open('../keys/api-keys.yaml', 'r') as f:
    API_KEYS = yaml.safe_load(f)